In [1]:
import tkinter as tk
from collections import deque
from math import sqrt      
        

#main class to create window, buttons etc
class GUI:
    def __init__(self):
        #window parameters
        root = tk.Tk()
        width = 800
        height = 600
        
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        
        center_x = int(screen_width/2 - width / 2)
        center_y = int(screen_height/2 - height*6/10)
        
        root.geometry(f'{width}x{height}+{center_x}+{center_y}')
        root.title('Bezier Curve Algorithm')
        root.rowconfigure(0, weight=1)
        root.columnconfigure(0, weight=1)

        self.canvas = canvas = tk.Canvas(root)
        canvas.grid(row=0, column=0, sticky = 'nsew')
        
        self.dots, self.middots, self.groups = [], [], []
        
        self.drag_item = self.hull = self.curve = self.spot = None
        
         # click drag a dot
        canvas.tag_bind("dot", "<ButtonPress-1>", self.dot_pressed) #self.dot_press
        #! canvas.tag_bind("dot", "<B1-Motion>", self.dot_motion)
        #! canvas.tag_bind("dot", "<ButtonRelease-1>", self.dot_release)
        
        
        #! canvas.bind("<ButtonPress-1>", self.add_dot)
        #! canvas.bind("<ButtonRelease-1>", lambda evt: self.draw_hull())
        
        
        frame = tk.Frame(root, bd = 1, relief = tk.SUNKEN)
        frame.grid(row = 1, column = 0, sticky = 'nsew')

        # Button to add new Dots to the canvas
        self.addButton = tk.Button(frame, text="Add", relief="raised",)  #command=self.add_dot_cb
        self.addButton.pack(side=tk.LEFT)
        self.addButton.state = True

        #! tk.Button(frame, text="Delete", command=self.delete_cb).pack(side=tk.LEFT)
        #! tk.Button(frame, text="Clear", command=self.clear_cb).pack(side=tk.LEFT)

        """"        self.anim_btn = tk.Button(frame, text="Animate", command=self.animate)
        self.anim_btn.pack(side=tk.LEFT)
        self.anim_btn.state = False
        """


        

        root.mainloop()
        
    def dot_pressed(self, event):
        print("hello")
        
    
    def add_dot(self, event):
        if not self.addButton.state:
            return
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y) 
        color = "black" if self.dots else "blue"
        self.dots.append(Dot(self.canvas, x, y, color, rad=4, tags="dot"))

   
  
    

 
   
   
  
if __name__ == "__main__":
    GUI()

In [28]:
2

In [4]:
import tkinter as tk
from collections import deque
from math import sqrt

def integrate(func, xlo, xhi, err_max):
    h = (xhi - xlo ) / 4.0
    xt = (xlo, (xhi + xlo ) / 2.0, xhi)
    yt = tuple(func(x) for x in xt)
    stack = deque([(xt, yt, h, err_max)])
    sigma = err = 0
    while stack:
        xt, yt, h, err_max = stack.popleft()
        x1, x3, x5 = xt
        y1, y3, y5 = yt
        while True:
            x2, x4 = x1 + h, x5 - h
            y2, y4 = func(x2), func(x4)
            s1 = y1 + y5
            s2 = 4.0 * (y2 + y4)

            # Get error from the absolute difference between
            # the 3-point & 5-point approximations
            err1 = abs((s1 - s2 + 6.0 * y3) * h / 45.0)
            if err1 < err_max:
                sigma += (s1 + s2 + 2.0 * y3) * h / 3.0
                err += err1
                #count += 1
                break

            h *= 0.5
            err_max *= 0.5

            # Make the left points 1, 2, 3 the new 1, 3, 5
            # and put the right points on the stack
            stack.append([(x3, x4, x5), (y3, y4, y5), h, err_max])
            x3, x5 = x2, x3
            y3, y5 = y2, y3
    return sigma, err 




def bezier(p, x0, y0, x1, y1, x2, y2, x3, y3):
    q = 1 - p
    t0 = q * q * q
    t1 = q * q * p
    t2 = q * p * p
    t3 = p * p * p
    x = t0 * x0 + 3 * t1 * x1 + 3 * t2 * x2 + t3 * x3
    y = t0 * y0 + 3 * t1 * y1 + 3 * t2 * y2 + t3 * y3
    return x, y




def flat(x0, y0, x1, y1, tol):
    return abs(x0*y1 - x1*y0) < tol * abs(x0 * x1 + y0 * y1)

def bezier_points(x0, y0, x1, y1, x2, y2, x3, y3, tol=0.001):
    if (flat(x1-x0, y1-y0, x2-x0, y2-y0, tol) and
        flat(x2-x1, y2-y1, x3-x1, y3-y1, tol)):
        return [x0, y0, x3, y3]

    x01, y01 = (x0 + x1) / 2., (y0 + y1) / 2.
    x12, y12 = (x1 + x2) / 2., (y1 + y2) / 2.
    x23, y23 = (x2 + x3) / 2., (y2 + y3) / 2.
    xa, ya = (x01 + x12) / 2., (y01 + y12) / 2.
    xb, yb = (x12 + x23) / 2., (y12 + y23) / 2.
    xc, yc = (xa + xb) / 2., (ya + yb) / 2.

    tol = 2. / (1. / tol - tol)
    return (bezier_points(x0, y0, x01, y01, xa, ya, xc, yc, tol)[:-2] + 
        bezier_points(xc, yc, xb, yb, x23, y23, x3, y3, tol))

class Dot:
    dots = {}
    def __init__(self, canvas, x, y, color, rad=1, tags=""):
        self.x, self.y = x, y
        self.canvas = canvas
        self.weight = 0
        self.id = canvas.create_oval(x-rad, y-rad, x+rad, y+rad,
            outline=color, fill=color, tags=tags)
        Dot.dots[self.id] = self

    def update(self, x, y):
        dx, dy = x - self.x, y - self.y
        self.x, self.y = x, y
        self.canvas.move(self.id, dx, dy)

    def __repr__(self):
        return f'Dot({self.id}, {self.x}, {self.y})'
    
    
    """"

    def delete(self):
        self.canvas.delete(self.id)
        del Dot.dots[self.id]
        
        
        """
        

class GUI: 
    #//*
    def __init__(self):
        root = tk.Tk()
        width = 800
        height = 600
        
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        
        center_x = int(screen_width/2 - width / 2)
        center_y = int(screen_height/2 - height*6/10)
        
        root.geometry(f'{width}x{height}+{center_x}+{center_y}')
        root.title('Bezier Curve Algorithm')
        root.rowconfigure(0, weight=1)
        root.columnconfigure(0, weight=1)


        # The Bezier control points, and groups for each curve
        self.dots, self.middots, self.groups = [], [], []

        # Various Canvas-related items: the current Dot being dragged, the
        # lines comprising the convex hull, the lines of the Bezier curve
        # and the animated spot.
        self.drag_item = self.hull = self.curve = self.spot = None

        # Add bindings for clicking & dragging any object with the "dot" tag
        canvas.tag_bind("dot", "<ButtonPress-1>", self.dot_press)
        canvas.tag_bind("dot", "<B1-Motion>", self.dot_motion)
        canvas.tag_bind("dot", "<ButtonRelease-1>", self.dot_release)
    #*//

        canvas.tag_bind("mdot", "<ButtonPress-1>", self.mdot_press)
        canvas.tag_bind("mdot", "<B1-Motion>", self.mdot_motion)
        canvas.tag_bind("mdot", "<ButtonRelease-1>", self.dot_release)

    #//*
        canvas.bind("<ButtonPress-1>", self.add_dot)
        canvas.bind("<ButtonRelease-1>", lambda evt: self.draw_hull())
    #*//

        frame = tk.Frame(root, bd=1, relief=tk.SUNKEN)
        frame.grid(row=1, column=0, sticky='nsew')

        # Button to add new Dots to the canvas
        self.add_btn = tk.Button(frame, text="Add", relief="sunken", 
            command=self.add_dot_cb)
        self.add_btn.pack(side=tk.LEFT)
        self.add_btn.state = True
        
        
        tk.Button(frame, text="Delete", command=self.delete_cb).pack(side=tk.LEFT)
        tk.Button(frame, text="Clear", command=self.clear_cb).pack(side=tk.LEFT)

        self.anim_btn = tk.Button(frame, text="Animate", command=self.animate)
        self.anim_btn.pack(side=tk.LEFT)
        self.anim_btn.state = False

        root.mainloop()

    def add_dot_cb(self):
        self.add_btn.state ^= 1
        self.add_btn["relief"] = "sunken" if self.add_btn.state else "raised"

        
        
        
#//*
    def add_dot(self, event):
        if not self.add_btn.state:
            return
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y) 
        color = "black" if self.dots else "blue"
        self.dots.append(Dot(self.canvas, x, y, color, rad=4, tags="dot"))
#*//
    


    
    def clear_cb(self):
        for dot in self.dots + self.middots:
            dot.delete()
        self.dots.clear()
        self.middots.clear()

        if self.hull:
            self.canvas.delete(self.hull)
            self.hull = None
        if self.curve:
            self.canvas.delete(self.curve)
            self.curve = None
        if not self.add_btn.state:
            self.add_dot_cb()

    def delete_cb(self):
        self.dots.pop().delete()
        if len(self.dots) % 2:
            self.middots.pop().delete()
        self.draw_hull()
    


#//*
    def dot_press(self, event):
        ''' Beginning drag of a dot '''
        # Record the item being dragged
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)
        self.drag_item = Dot.dots[self.canvas.find_closest(x, y)[0]]
#*//

    def dot_motion(self, event):
        ''' Handle dragging of a dot '''
        if self.drag_item is None:
            return
        # Get the mouse location
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y) 
        # Move the Dot to the new location
        self.drag_item.update(x, y)
        self.draw_hull()

    def mdot_press(self, event):
        ''' Beginning drag of an mdot '''
        # Record the item being dragged
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)
        self.drag_item = Dot.dots[self.canvas.find_closest(x, y)[0]]

        # Find the control points of the segment containing this mdot
        idx = self.middots.index(self.drag_item) * 2
        d0, d1 = self.dots[idx:idx+2]
        # Create a function to find the weight of the closest point
        # on the segment to (x, y)
        x0, y0, x1, y1 = d0.x, d0.y, d1.x, d1.y
        dx, dy = x1 - x0, y1 - y0
        d2 = dx ** 2 + dy ** 2
        def get_weight(x, y):
            # Use the dot product to find the weight of the closest
            # point between (x0, y0) and (x1, y1) to (x, y)
            w = (dx * (x - x0) + dy * (y - y0)) / d2
            # Bound it to inside the segment
            return min(max(0.001, w), 0.999)
        self.get_weight = get_weight

    def mdot_motion(self, event):
        ''' Handle dragging of an mdot object '''
        if self.drag_item is None:
            return
        # Get the mouse location
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)
        # Set the weight of the closest point on the bounding segment
        self.drag_item.weight = self.get_weight(x, y)
        self.draw_hull()

    def dot_release(self, event):
        ''' End drag of an object '''
        # Reset the drag information
        self.drag_item = None

    def draw_hull(self):
        ''' Draw the convex hull of the Bezier curve '''
        if len(self.dots) < 2:
            return

        # Create a flat list of the control point coords
        coords = []
        for dot in self.dots:
            coords.extend((dot.x, dot.y))
        if len(coords) % 4 == 0:
            coords.extend(coords[:2])

        if self.hull:
            self.canvas.coords(self.hull, coords)
        else:
            self.hull = self.canvas.create_line(coords, fill="grey", dash=[4, 4])

        self.draw_curve()

    def draw_curve(self):
        if len(self.dots) < 4:
            return

        # Place the midpoints, which are actually the start
        # start and end points of each Bezier section
        middots = self.middots
        dot_queue = deque(middots)

        coords = []
        for i, dot in enumerate(self.dots):
            x, y = dot.x, dot.y
            if i % 2:
                px, py = coords[-2:]
                if dot_queue:
                    mdot = dot_queue.popleft()
                    w0, w1 = mdot.weight, 1 - mdot.weight
                    mx, my = px*w1 + x*w0, py*w1 + y*w0
                    mdot.update(mx, my)
                else:
                    w0 = w1 = 0.5
                    mx, my = px*w1 + x*w0, py*w1 + y*w0
                    mdot = Dot(self.canvas, mx, my, "white", tags="mdot", rad=4)
                    mdot.weight = w0
                    middots.append(mdot)
                coords.extend((mx, my))
            coords.extend((x, y))

        coords.extend(coords[:4])

        # Draw the Bezier curves
        points = []
        #! self.groups.clear()
        maxi = (len(coords) - 2) // 6 * 6 + 2
        for i in range(2, maxi, 6):
            group = coords[i:i+8]
            if len(group) < 8: 
                break
            points.extend(bezier_points(*group)[:-2])
            self.groups.append(group)
        points.extend(coords[2:4])

        if self.curve:
            self.canvas.coords(self.curve, points)
        else:
            self.curve = self.canvas.create_line(points, fill="blue")

    def anim_spot(self, gen):
        ''' Callback loop for the animated Dot '''
        try:
            self.spot.update(*next(gen))
        except (AttributeError, StopIteration):
            return
        self.canvas.after(10, lambda: self.anim_spot(gen))

    def animate(self):
        ''' Animate a Dot moving along the Bezier curve at constant speed '''
        state = self.anim_btn.state = self.anim_btn.state ^ 1
        self.anim_btn["relief"] = "sunken" if state else "raised"
        if self.spot or not state:
            return

        start = self.middots[0]
        self.spot = Dot(self.canvas, start.x, start.y, "red", rad=4, tags="spot")
        self.anim_spot(self.do_spot())

    def do_spot(self):
        while True:
            for group in self.groups:
                yield from self.anim_group(group)
            if not self.anim_btn.state:
                self.spot = self.spot.delete()
                return

    def anim_group(self, group):
        ''' A generator that calculates the animated Dot coordinates '''
        x0, y0, x1, y1, x2, y2, x3, y3 = group
        u1, v1 = x1 - x0, y1 - y0
        u2, v2 = x2 - x1, y2 - y1
        u3, v3 = x3 - x2, y3 - y2

        def bezier_diff(p):
            ''' The arclength differential of the Bezier curve '''
            q = 1 - p
            t1 = q * q
            t2 = q * p
            t3 = p * p
            dx = 3 * t1 * u1 + 6 * t2 * u2 + 3 * t3 * u3
            dy = 3 * t1 * v1 + 6 * t2 * v2 + 3 * t3 * v3
            return sqrt(dx*dx + dy*dy)

        steplen = 2.0
        old_p = p = 0.0
        #vlo, vhi = 10 * steplen, 0.0
        while self.anim_btn.state and p < 1.0:
            # Estimate next p
            p = old_p + steplen / bezier_diff(old_p)
            # Refine it using Newton's Method
            for j in range(9):
                # Get the distance along the curve from old_p to p
                a, err = integrate(bezier_diff, old_p, p, 1e-5)
                delta = steplen - a
                p += delta / bezier_diff(p)
                if abs(delta) < 0.01:
                    break

            # Measure the actual step length
            #a, err = integrate(bezier_diff, old_p, p, 1e-6)
            #vlo, vhi = min(vlo, a), max(vhi, a)

            yield bezier(p, x0, y0, x1, y1, x2, y2, x3, y3)
            old_p = p
            
if __name__ == "__main__":
    GUI()

NameError: name 'canvas' is not defined

In [ ]:
3

In [3]:
import tkinter as tk
from collections import deque
from math import sqrt

def integrate(func, xlo, xhi, err_max):
    h = (xhi - xlo ) / 4.0
    xt = (xlo, (xhi + xlo ) / 2.0, xhi)
    yt = tuple(func(x) for x in xt)
    stack = deque([(xt, yt, h, err_max)])
    sigma = err = 0
    while stack:
        xt, yt, h, err_max = stack.popleft()
        x1, x3, x5 = xt
        y1, y3, y5 = yt
        while True:
            x2, x4 = x1 + h, x5 - h
            y2, y4 = func(x2), func(x4)
            s1 = y1 + y5
            s2 = 4.0 * (y2 + y4)

    
            err1 = abs((s1 - s2 + 6.0 * y3) * h / 45.0)
            if err1 < err_max:
                sigma += (s1 + s2 + 2.0 * y3) * h / 3.0
                err += err1
                #count += 1
                break

            h *= 0.5
            err_max *= 0.5

            # Make the left points 1, 2, 3 the new 1, 3, 5
            # and put the right points on the stack
            stack.append([(x3, x4, x5), (y3, y4, y5), h, err_max])
            x3, x5 = x2, x3
            y3, y5 = y2, y3
    return sigma, err 





def bezier(p, x0, y0, x1, y1, x2, y2, x3, y3):
    q = 1 - p
    t0 = q * q * q
    t1 = q * q * p
    t2 = q * p * p
    t3 = p * p * p
    x = t0 * x0 + 3 * t1 * x1 + 3 * t2 * x2 + t3 * x3
    y = t0 * y0 + 3 * t1 * y1 + 3 * t2 * y2 + t3 * y3
    return x, y




def flat(x0, y0, x1, y1, tol):
    return abs(x0*y1 - x1*y0) < tol * abs(x0 * x1 + y0 * y1)

def bezier_points(x0, y0, x1, y1, x2, y2, x3, y3, tol=0.001):
    if (flat(x1-x0, y1-y0, x2-x0, y2-y0, tol) and
        flat(x2-x1, y2-y1, x3-x1, y3-y1, tol)):
        return [x0, y0, x3, y3]

    x01, y01 = (x0 + x1) / 2., (y0 + y1) / 2.
    x12, y12 = (x1 + x2) / 2., (y1 + y2) / 2.
    x23, y23 = (x2 + x3) / 2., (y2 + y3) / 2.
    xa, ya = (x01 + x12) / 2., (y01 + y12) / 2.
    xb, yb = (x12 + x23) / 2., (y12 + y23) / 2.
    xc, yc = (xa + xb) / 2., (ya + yb) / 2.

    tol = 2. / (1. / tol - tol)
    return (bezier_points(x0, y0, x01, y01, xa, ya, xc, yc, tol)[:-2] + 
        bezier_points(xc, yc, xb, yb, x23, y23, x3, y3, tol))

class Dot:
    dots = {}
    def __init__(self, canvas, x, y, color, rad=1, tags=""):
        self.x, self.y = x, y
        self.canvas = canvas
        self.weight = 0
        self.id = canvas.create_oval(x-rad, y-rad, x+rad, y+rad,
            outline=color, fill=color, tags=tags)
        Dot.dots[self.id] = self

    def update(self, x, y):
        dx, dy = x - self.x, y - self.y
        self.x, self.y = x, y
        self.canvas.move(self.id, dx, dy)

    def __repr__(self):
        return f'Dot({self.id}, {self.x}, {self.y})'
    
    
        

class GUI: 
    #//*
    def __init__(self):
        root = tk.Tk()
        width = root.winfo_screenwidth() * 4 // 5
        height = root.winfo_screenheight() * 4 // 5
        root.geometry(f'{width}x{height}')
        root.title('Bezier Track')
        root.rowconfigure(0, weight=1)
        root.columnconfigure(0, weight=1)

        self.canvas = canvas = tk.Canvas(root)
        canvas.grid(row=0, column=0, sticky='nsew')
        
    

        # The Bezier control points, and groups for each curve
        self.dots, self.middots, self.groups = [], [], []

        # Various Canvas-related items: the current Dot being dragged, the
        # lines comprising the convex hull, the lines of the Bezier curve
        # and the animated spot.
        self.drag_item = self.hull = self.curve = self.spot = None

        # Add bindings for clicking & dragging any object with the "dot" tag
        canvas.tag_bind("dot", "<ButtonPress-1>", self.dot_press)
        canvas.tag_bind("dot", "<B1-Motion>", self.dot_motion)
        canvas.tag_bind("dot", "<ButtonRelease-1>", self.dot_release)
    #*//

        canvas.tag_bind("mdot", "<ButtonPress-1>", self.mdot_press)
        canvas.tag_bind("mdot", "<B1-Motion>", self.mdot_motion)
        canvas.tag_bind("mdot", "<ButtonRelease-1>", self.dot_release)

    #//*
        canvas.bind("<ButtonPress-1>", self.add_dot)
        canvas.bind("<ButtonRelease-1>", lambda evt: self.draw_hull())
    #*//

        frame = tk.Frame(root, bd=1, relief=tk.SUNKEN)
        frame.grid(row=1, column=0, sticky='nsew')

        # Button to add new Dots to the canvas
        self.add_btn = tk.Button(frame, text="Add", relief="sunken", 
            command=self.add_dot_cb)
        self.add_btn.pack(side=tk.LEFT)
        self.add_btn.state = True
        
       
        self.anim_btn = tk.Button(frame, text="Animate", command=self.animate)
        self.anim_btn.pack(side=tk.LEFT)
        self.anim_btn.state = False

        root.mainloop()

    def add_dot_cb(self):
        self.add_btn.state ^= 1
        self.add_btn["relief"] = "sunken" if self.add_btn.state else "raised"

        
        
        
#//*
    def add_dot(self, event):
        if not self.add_btn.state:
            return
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y) 
        color = "black" if self.dots else "blue"
        self.dots.append(Dot(self.canvas, x, y, color, rad=4, tags="dot"))
#*//
    


    
 


    def dot_press(self, event):
        ''' Beginning drag of a dot '''
        # Record the item being dragged
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)
        self.drag_item = Dot.dots[self.canvas.find_closest(x, y)[0]]

        
        
        
    def dot_motion(self, event):
        ''' Handle dragging of a dot '''
        if self.drag_item is None:
            return
        # Get the mouse location
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y) 
        # Move the Dot to the new location
        self.drag_item.update(x, y)
        self.draw_hull()

    def mdot_press(self, event):
        ''' Beginning drag of an mdot '''
        # Record the item being dragged
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)
        self.drag_item = Dot.dots[self.canvas.find_closest(x, y)[0]]

        # Find the control points of the segment containing this mdot
        idx = self.middots.index(self.drag_item) * 2
        d0, d1 = self.dots[idx:idx+2]
        # Create a function to find the weight of the closest point
        # on the segment to (x, y)
        x0, y0, x1, y1 = d0.x, d0.y, d1.x, d1.y
        dx, dy = x1 - x0, y1 - y0
        d2 = dx ** 2 + dy ** 2
        def get_weight(x, y):
            # Use the dot product to find the weight of the closest
            # point between (x0, y0) and (x1, y1) to (x, y)
            w = (dx * (x - x0) + dy * (y - y0)) / d2
            # Bound it to inside the segment
            return min(max(0.001, w), 0.999)
        self.get_weight = get_weight

    def mdot_motion(self, event):
        ''' Handle dragging of an mdot object '''
        if self.drag_item is None:
            return
        # Get the mouse location
        x, y = self.canvas.canvasx(event.x), self.canvas.canvasy(event.y)
        # Set the weight of the closest point on the bounding segment
        self.drag_item.weight = self.get_weight(x, y)
        self.draw_hull()

    def dot_release(self, event):
        self.drag_item = None
        
    
    def draw_hull(self):
        ''' Draw the convex hull of the Bezier curve '''
        if len(self.dots) < 2:
            return

        
        if len(self.dots) < 2:
            return

        # Create a flat list of the control point coords
        coords = []
        for dot in self.dots:
            coords.extend((dot.x, dot.y))
        if len(coords) % 4 == 0:
            coords.extend(coords[:2])

        if self.hull:
            self.canvas.coords(self.hull, coords)
        else:
            self.hull = self.canvas.create_line(coords, fill="grey", dash=[4, 4])

        self.draw_curve()

    def draw_curve(self):
        if len(self.dots) < 4:
            return

        # Place the midpoints, which are actually the start
        # start and end points of each Bezier section
        middots = self.middots
        dot_queue = deque(middots)

        coords = []
        for i, dot in enumerate(self.dots):
            x, y = dot.x, dot.y
            if i % 2:
                px, py = coords[-2:]
                if dot_queue:
                    mdot = dot_queue.popleft()
                    w0, w1 = mdot.weight, 1 - mdot.weight
                    mx, my = px*w1 + x*w0, py*w1 + y*w0
                    mdot.update(mx, my)
                else:
                    w0 = w1 = 0.5
                    mx, my = px*w1 + x*w0, py*w1 + y*w0
                    mdot = Dot(self.canvas, mx, my, "white", tags="mdot", rad=4)
                    mdot.weight = w0
                    middots.append(mdot)
                coords.extend((mx, my))
            coords.extend((x, y))

        coords.extend(coords[:4])

        # Draw the Bezier curves
        points = []
        maxi = (len(coords) - 2) // 6 * 6 + 2
        
        for i in range(2, maxi, 6):
            group = coords[i:i+8]
            if len(group) < 8: 
                break
            points.extend(bezier_points(*group)[:-2])
            self.groups.append(group)
        points.extend(coords[2:4])

        if self.curve:
            self.canvas.coords(self.curve, points)
        else:
            self.curve = self.canvas.create_line(points, fill="blue")

    def anim_spot(self, gen):
        ''' Callback loop for the animated Dot '''
        try:
            self.spot.update(*next(gen))
        except (AttributeError, StopIteration):
            return
        self.canvas.after(10, lambda: self.anim_spot(gen))

    def animate(self):
        ''' Animate a Dot moving along the Bezier curve at constant speed '''
        state = self.anim_btn.state = self.anim_btn.state ^ 1
        self.anim_btn["relief"] = "sunken" if state else "raised"
        if self.spot or not state:
            return

        start = self.middots[0]
        self.spot = Dot(self.canvas, start.x, start.y, "red", rad=4, tags="spot")
        self.anim_spot(self.do_spot())

    def do_spot(self):
        while True:
            for group in self.groups:
                yield from self.anim_group(group)
            if not self.anim_btn.state:
                self.spot = self.spot.delete()
                return

    def anim_group(self, group):
        ''' A generator that calculates the animated Dot coordinates '''
        x0, y0, x1, y1, x2, y2, x3, y3 = group
        u1, v1 = x1 - x0, y1 - y0
        u2, v2 = x2 - x1, y2 - y1
        u3, v3 = x3 - x2, y3 - y2

        def bezier_diff(p):
            ''' The arclength differential of the Bezier curve '''
            q = 1 - p
            t1 = q * q
            t2 = q * p
            t3 = p * p
            dx = 3 * t1 * u1 + 6 * t2 * u2 + 3 * t3 * u3
            dy = 3 * t1 * v1 + 6 * t2 * v2 + 3 * t3 * v3
            return sqrt(dx*dx + dy*dy)

        steplen = 2.0
        old_p = p = 0.0
        #vlo, vhi = 10 * steplen, 0.0
        while self.anim_btn.state and p < 1.0:
            # Estimate next p
            p = old_p + steplen / bezier_diff(old_p)
            # Refine it using Newton's Method
            for j in range(9):
                # Get the distance along the curve from old_p to p
                a, err = integrate(bezier_diff, old_p, p, 1e-5)
                delta = steplen - a
                p += delta / bezier_diff(p)
                if abs(delta) < 0.01:
                    break

           

            yield bezier(p, x0, y0, x1, y1, x2, y2, x3, y3)
            old_p = p
            
if __name__ == "__main__":
    GUI()

In [ ]:
def curveCreation(x0, y0,x1,y1, x2,y2, x3, y3, x4, y4, x5, y5, x6, y6, x7, y7):
    xCoordinates = []
    yCoordinates = []
    t = 0
    ts = 0.01
    
    #
    
    
    while t <= 1:
        B0 = (1-t)*(1-t)*(1-t)*(1-t)*(1-t)*(1-t)*(1-t)
        B1 = 7*t*(1-t)*(1-t)*(1-t)*(1-t)*(1-t)*(1-t)
        B2 = 21*t*t*(1-t)*(1-t)*(1-t)*(1-t)*(1-t)
        B3 = 35*t*t*t*(1-t)*(1-t)*(1-t)*(1-t)
        B4 = 35*t*t*t*t*(1-t)*(1-t)*(1-t)
        B5 = 21*t*t*t*t*t*(1-t)*(1-t)
        B6 = 7*t*t*t*t*t*t*(1-t)
        B7 = t*t*t*t*t*t*t
        
        x = int(x0*B0 + x1*B1 + x2*B2 + x3*B3 + x4*B4 + x5*B5 + x6*B6 + x7*B7) 
        y = int(y0*B0 + y1*B1 + y2*B2 + y3*B3 + y4*B4 + y5*y5 + y6*B6 + y7*B7)
        
                
         
        t=t+ts
        
        xCoordinates.append(x)
        yCoordinates.append(y)
        
    plt.plot(xCoordinates, yCoordinates)
                  
        
    return         
curveCreation 

In [ ]:
FINAL BUT NO

In [ ]:
import time
import tkinter as tk


class Example(tk.Frame):

    def __init__(self, parent):
        tk.Frame.__init__(self, parent)

        # create a canvas
        self.canvas = tk.Canvas(width=800, height=800)
        self.canvas.pack(fill="both", expand=True)
        self.pointscoords = []

        # this data is used to keep track of an
        # item being dragged
        self._drag_data = {"x": 0, "y": 0, "item": None}

        # on mouse right click create a token on the canvas with click coordinates
        self.canvas.bind("<Button-3>", lambda event: self.create_token(event.x, event.y))
        self.canvas.bind("<Button-2>", lambda event: self.draw_bezier())

        # add bindings for clicking, dragging and releasing over
        # any object with the "token" tag
        self.canvas.tag_bind("token", "<ButtonPress-1>", self.drag_start)
        self.canvas.tag_bind("token", "<ButtonRelease-1>", self.drag_stop)
        self.canvas.tag_bind("token", "<B1-Motion>", self.drag)

    def create_token(self, x, y):
        if (len(self.pointscoords) < 4):
            """Create a token at the given coordinate in the given color"""
            self.canvas.create_oval(
                x - 10,
                y - 10,
                x + 10,
                y + 10,
                outline="black",
                fill="blue",
                tags=("token",),
            )
            self.pointscoords.append((x, y))

            if (len(self.pointscoords) == 4):
                self.clear_beizer()
                self.beizer_curved_line_for_4_points()
            if (len(self.pointscoords) == 3):
                self.clear_beizer()
                self.bezier_curve_3()

    def drag_start(self, event):
        """Begining drag of an object"""
        # record the item and its location
        self._drag_data["item"] = self.canvas.find_closest(event.x, event.y)[0]
        self._drag_data["x"] = event.x
        self._drag_data["y"] = event.y

    def drag_stop(self, event):
        """End drag of an object"""
        # reset the drag information
        self._drag_data["item"] = None
        self._drag_data["x"] = 0
        self._drag_data["y"] = 0
        if len(self.pointscoords) > 3:
            self.clear_beizer()
            self.draw_bezier()
        if len(self.pointscoords) == 3:
            self.clear_beizer()
            self.bezier_curve_3()

    def drag(self, event):
        """Handle dragging of an object"""
        # compute how much the mouse has moved
        delta_x = event.x - self._drag_data["x"]
        delta_y = event.y - self._drag_data["y"]
        # move the object the appropriate amount
        self.canvas.move(self._drag_data["item"], delta_x, delta_y)
        # record the new position
        self._drag_data["x"] = event.x
        self._drag_data["y"] = event.y

        # change point coords on move
        self.setCoords(event.x - delta_x, event.y - delta_y, event.x, event.y)
        print(self.pointscoords)
        if event.x < 1 or event.y < 1 or event.x > 800 or event.y > 800:
            self.canvas.delete(self._drag_data["item"])
            print("deleted")
        print(event.x, event.y)

    def setCoords(self, xold, yold, xnew, ynew):
        for i in range(xold - 10, xold + 10):
            for j in range(yold - 10, yold + 10):
                for p in self.pointscoords:
                    if p[0] == i and p[1] == j:
                        index = self.pointscoords.index(p)
                        self.pointscoords[index] = (xnew, ynew)
                        print(self.pointscoords)
                        break

    def beizer_curved_line_for_4_points(self):
        coords = []
        x0, y0 = self.pointscoords[0]
        x1, y1 = self.pointscoords[1]
        x2, y2 = self.pointscoords[2]
        x3, y3 = self.pointscoords[3]
        for t in range(0, 1000):
            t /= 1000
            x = (1 - t) ** 3 * x0 + 3 * t * (1 - t) ** 2 * x1 + 3 * t ** 2 * (1 - t) * x2 + t ** 3 * x3
            y = (1 - t) ** 3 * y0 + 3 * t * (1 - t) ** 2 * y1 + 3 * t ** 2 * (1 - t) * y2 + t ** 3 * y3
            coords.append((x, y))
            self.canvas.create_oval(x, y, x, y, outline="red", fill="red")
        oval = self.canvas.create_oval(coords[0][0] - 5, coords[0][1] - 5, coords[1][0] + 5, coords[1][1] + 5,
                                       outline="", fill="blue", width=20)
        for i in range(1, len(coords)):
            print(coords[i])
            self.canvas.move(oval, coords[i][0] - coords[i - 1][0], coords[i][1] - coords[i - 1][1])
            self.canvas.update()
        self.canvas.delete(oval)
        return coords

    # beizer curve for 3 points
    def bezier_curve_3(self):
        coords = []
        x0, y0 = self.pointscoords[0]
        x1, y1 = self.pointscoords[1]
        x2, y2 = self.pointscoords[2]
        for t in range(0, 1000):
            t /= 1000
            x = (1-t)**2 * x0 + 2 * (1-t) * t * x1 + t**2 * x2
            y = (1-t)**2 * y0 + 2 * (1-t) * t * y1 + t**2 * y2
            coords.append((x, y))
            self.canvas.create_oval(x, y, x, y, outline="red", fill="red")
        oval = self.canvas.create_oval(coords[0][0] - 5, coords[0][1] - 5, coords[1][0] + 5, coords[1][1] + 5, outline="", fill="blue", width=20)
        for i in range(1, len(coords)):
            print(coords[i])
            self.canvas.move(oval, coords[i][0] - coords[i - 1][0], coords[i][1] - coords[i - 1][1])
            self.canvas.update()
        self.canvas.delete(oval)
        return coords

    def clear_beizer(self):
        self.canvas.delete("all")
        for x, y in self.pointscoords:
            self.canvas.create_oval(x - 10, y - 10, x + 10, y + 10, outline="black", fill="red", tags=("token",))

            

if __name__ == "__main__":
    root = tk.Tk()
    Example(root).pack(fill="both", expand=True)
    root.mainloop()
